Trampush Comp Statistic Code

In [ ]:
function CV_all = CompStatSpace(Topo, n)
% Calulate CV values from chronostratigraphic surfaces mapped in space.
% Created by Sheila Trampush, Last updated 2016-08-26. 
% Surface data (Topo) must be evenly spaced, in chronostratigraphic order, 
% and the matrix must only contain elevations, with lateral position
% indicated by row number and time/surface number by column number. 
% Produces matrix with 4 fields: (1) mean thickness between 2 surfaces,
% (2)CV value for the two surfaces, (3) index number of first surface used, 
% and (4) index number of the second surface used. 

A=~isnan(Topo);

max_thick = nanmean(nanmax(Topo, [], 2)-nanmin(Topo,[],2)); %mean thickness of dataset
eta_mean = max_thick/mean(sum(A,2)); %expected thickness between two surfaces
exp_pair_no=n*(n-1)/2; %expected number of unique pairs of surfaces
CV_all=nan(exp_pair_no,4); %initialize matrix for CV/eta pairs
pair=1; % initialize counter for the number of pairs 

for i=1:n %loop through all surfaces 
   for j=1:n-1 %loop through all surfaces 
      if i+j>n %make sure it isn't looping through surfaces that don't exist
          break
      end
      surf1=Topo(:,j); %lower surface
      surf2=Topo(:, j+i); %surface above lower surface 
      diffTopo=surf2-surf1; %difference in elevation for each x 
      Positive = diffTopo > 0; %make sure that negative or zero values can't occur (I assume they are digitizing errors)
      diffTopo(~Positive) = NaN;%replace values with nans to prevent negative values 
      rat_diffTopo=diffTopo/(eta_mean*i); %ratio of observed sedimentation to expected sedimentation
      if nanstd(rat_diffTopo)>1e-5 %sum(Positive) > 2 %how many x nodes should overlap? 
            CV_all(pair,2)=nanstd(rat_diffTopo); %CV--standard deviation of ratio observed to expected (~sigma_ss)
            CV_all(pair, 1)=nanmean(diffTopo); %eta-mean stratigraphic thickness measured (~time/measurement window)
      else
            CV_all(pair,2)=NaN; %CV--standard deviation of ratio observed to expected (~sigma_ss)
            CV_all(pair, 1)=NaN; %eta-mean stratigraphic thickness measured (~time/measurement window)
      end 
      CV_all(pair,3)=j; %to keep track of which CV pair belongs with which pair of surfaces 
      CV_all(pair,4)=j+i; %to keep track of which CV pair belongs with which pair of surfaces 
      pair=pair+1; %update counter for next loop iteration
      
   end
end
end


CompStatLogBin


In [ ]:
function CV_LogBinned = CompStatLogBin(CV_all, N, minBin, maxBin)
% To bin CV data into log bins. Input needs to be from CompStatSpace.m.
% Created by Sheila Trampush at Penn State, last updated 2016-03-04. 

binLog=logspace(minBin,maxBin,N); %boundaries of bins
CV_LogBinned=nan(N,6); %initialize matrix
CV_all_sorted=sortrows(CV_all, 1); %sort all data points by strat thickness

for b=1:N-1
    
    [bin_temp_i, ~]=find(CV_all_sorted(:,1)>=binLog(b),1, 'first');
%     find first appearance of a datapoint that has thickness larger than
%     b, the minimum extent of bin.
    [bin_temp_I, ~]=find(CV_all_sorted(:,1)<=binLog(b+1),1, 'last');
%     find last appearance of a datpoint that has thickness smaller than
%     the maximum extent of bin.
    temp_CV = CV_all_sorted(bin_temp_i:bin_temp_I, :); 
%     if sorted right, the bin should be everything after the first
%     appearance of the minimum extent to the last appearance of the
%     maximum extent. 
    midpoint=(binLog(b)+binLog(b+1))/2; %place at center of bin
    [i, ~]=find(temp_CV(:,2)>0);
    temp_CV=temp_CV(i,:); %exclude CV below 0 
if numel(temp_CV)>2 %exclude bins with one datapoint 
    CV_LogBinned(b, :)=[midpoint, median(temp_CV(:,2)), ...
        prctile(temp_CV(:,2), 2.5), prctile(temp_CV(:,2), 97.5), ...
        (prctile(temp_CV(:,2), 97.5)-prctile(temp_CV(:,2), 5))/geomean(temp_CV(:,2)),...
        length(temp_CV)]; 
    %     final binned data matrix should have mean eta, mean CV, std eta,
    %     and std CV to enable both vertical and horizontal error bars
elseif numel(temp_CV)==2
    CV_LogBinned(b, :)=[temp_CV(:,1), temp_CV(:,2), 0, 0, 0, 1];
end

end
end